In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
from copy import deepcopy
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, stride, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * 4:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * 4, 1, stride, bias=False),
                nn.BatchNorm2d(planes * 4)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out)) + self.shortcut(x)
        return F.relu(out)

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], 2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = F.avg_pool2d(x, 4).view(x.size(0), -1)
        return self.linear(x)

def ResNet101(num_classes): return ResNet(Bottleneck, [3, 4, 23, 3], num_classes)

class MobileNetV2_Block(nn.Module):
    def __init__(self, in_planes, out_planes, expansion, stride):
        super().__init__()
        planes = expansion * in_planes
        self.conv1 = nn.Conv2d(in_planes, planes, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, stride, 1, groups=planes, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, out_planes, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes)
        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, 1, bias=False),
                nn.BatchNorm2d(out_planes)
            )
        self.stride = stride
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.stride == 1:
            out += self.shortcut(x)
        return out

class MobileNetV2(nn.Module):
    cfg = [(1, 16, 1, 1), (6, 24, 2, 1), (6, 32, 3, 2),
           (6, 64, 4, 2), (6, 96, 3, 1), (6, 160, 3, 2), (6, 320, 1, 1)]
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(32)
        self.conv2 = nn.Conv2d(320, 1280, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.linear = nn.Linear(1280, num_classes)
    def _make_layers(self, in_planes):
        layers = []
        for e, o, n, s in self.cfg:
            for i in range(n):
                layers.append(MobileNetV2_Block(in_planes, o, e, s if i == 0 else 1))
                in_planes = o
        return nn.Sequential(*layers)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layers(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.avg_pool2d(x, 4).view(x.size(0), -1)
        return self.linear(x)

vgg_cfg = {
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M',
              512, 512, 512, 'M', 512, 512, 512, 'M']
}
class VGG(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = self._make_layers(vgg_cfg["VGG16"])
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096), nn.ReLU(), nn.Dropout(),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.features(x).view(x.size(0), -1)
        return self.classifier(x)
    def _make_layers(self, cfg):
        layers = []; in_c = 3
        for x in cfg:
            if x == 'M': layers += [nn.MaxPool2d(2,2)]
            else:
                layers += [nn.Conv2d(in_c, x, 3, 1, 1), nn.BatchNorm2d(x), nn.ReLU()]
                in_c = x
        layers += [nn.AvgPool2d(1)]
        return nn.Sequential(*layers)


In [3]:
class pruning_engine_base:
    def __init__(self, pruning_ratio, pruning_method):
        self.pruning_ratio = 1 - pruning_ratio
        self.pruning_method = pruning_method
        self.mask_number = 0.0
        self.device = device

    def base_remove_filter_by_index(self, weight, remove_filter_idx, bias=None, mean=None, var=None, linear=False):
        with torch.no_grad():
            if mean is not None:
                for idx in remove_filter_idx:
                    weight[idx.item()] = self.mask_number
                    bias[idx.item()] = self.mask_number
                    mean[idx.item()] = self.mask_number
                    var[idx.item()] = self.mask_number
                return weight, bias, mean, var
            elif bias is not None:
                for idx in remove_filter_idx:
                    weight[idx.item()] = self.mask_number
                    bias[idx.item()] = self.mask_number
                return weight, bias
            else:
                for idx in remove_filter_idx:
                    weight[idx.item()] = self.mask_number
                return weight

    def base_remove_kernel_by_index(self, weight, remove_filter_idx, linear=False):
        with torch.no_grad():
            for idx in remove_filter_idx:
                weight[:, idx.item()] = self.mask_number
        return weight


class Taylor:
    def __init__(self, tool_net, taylor_loader, total_layer, total_sample_size):
        self.tool_net = tool_net
        self.taylor_loader = taylor_loader
        self.total_layer = total_layer
        self.total_sample_size = total_sample_size
        self.forward_activations = [None] * total_layer
        self.backward_gradients = [None] * total_layer

    def attach_hooks(self, conv_layers):
        def make_forward_hook(i):
            def forward_hook(module, inp, out):
                with torch.no_grad():
                    abs_out = out.detach().abs().sum(dim=0)
                    if self.forward_activations[i] is None:
                        self.forward_activations[i] = abs_out
                    else:
                        if self.forward_activations[i].shape != abs_out.shape:
                            self.forward_activations[i] = abs_out
                        else:
                            self.forward_activations[i] += abs_out
            return forward_hook

        def make_backward_hook(i):
            def backward_hook(module, grad_in, grad_out):
                with torch.no_grad():
                    grad = grad_out[0].abs().sum(dim=0)
                    if self.backward_gradients[i] is None:
                        self.backward_gradients[i] = grad
                    else:
                        if self.backward_gradients[i].shape != grad.shape:
                            self.backward_gradients[i] = grad
                        else:
                            self.backward_gradients[i] += grad
            return backward_hook

        for i, conv in enumerate(conv_layers):
            conv.register_forward_hook(make_forward_hook(i))
            conv.register_backward_hook(make_backward_hook(i))

    def Taylor_add_gradient(self):
        conv_layers = [m for m in self.tool_net.modules() if isinstance(m, nn.Conv2d)]
        assert len(conv_layers) == self.total_layer, "Mismatch in layer count."
        self.attach_hooks(conv_layers)
        optimizer = optim.Adam(self.tool_net.parameters())
        criterion = nn.CrossEntropyLoss()
        self.tool_net.train()
        with tqdm(total=len(self.taylor_loader)) as pbar:
            for inputs, labels in self.taylor_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self.tool_net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                pbar.update()
        for i in range(self.total_layer):
            if self.forward_activations[i] is not None:
                self.forward_activations[i] /= float(self.total_sample_size)
            if self.backward_gradients[i] is not None:
                self.backward_gradients[i] /= float(self.total_sample_size)

    def store_grad_layer(self, layer_store):
        for i, layer in enumerate(layer_store):
            layer.__dict__["feature_map"] = self.forward_activations[i]
            layer.__dict__["gradient"] = self.backward_gradients[i]

    def clear_stats(self):
        self.forward_activations = [None] * self.total_layer
        self.backward_gradients = [None] * self.total_layer

    def Taylor_pruning(self, layer):
        fm = layer.feature_map
        gr = layer.gradient
        importance = (fm * gr).abs().mean(dim=(1, 2))
        importance /= (importance.norm() + 1e-8)
        _, sorted_idx = torch.sort(importance, descending=True)
        return sorted_idx


class pruning_engine(pruning_engine_base):
    def __init__(self, pruning_method, pruning_ratio=0.0, individual=False,
                 conv_to_bn_map=None, **kwargs):
        super().__init__(pruning_ratio, pruning_method)
        self.conv_to_bn_map = conv_to_bn_map if conv_to_bn_map else {}
        self.remove_filter_idx_history = {"previous_layer": None, "current_layer": None}
        self.individual = individual
        if self.pruning_method == "Taylor":
            layer_store = kwargs["layer_store_private_variable"]
            self.taylor_helper = Taylor(
                tool_net=kwargs["tool_net"],
                taylor_loader=kwargs["taylor_loader"],
                total_layer=len(layer_store),
                total_sample_size=kwargs["total_sample_size"]
            )
            self.taylor_helper.clear_stats()
            self.taylor_helper.Taylor_add_gradient()
            self.taylor_helper.store_grad_layer(layer_store)
            self.pruning_criterion = self.taylor_helper.Taylor_pruning

    def set_layer(self, layer, main_layer=False):
        self.copy_layer = deepcopy(layer)
        if main_layer:
            if self.individual:
                self.remove_filter_idx_history = {"previous_layer": None, "current_layer": None}
            self.remove_filter_idx_history["previous_layer"] = self.remove_filter_idx_history["current_layer"]
            self.remove_filter_idx_history["current_layer"] = None
            remove_filter_idx = self.pruning_criterion(self.copy_layer)
            num_prune = int(len(remove_filter_idx) * self.pruning_ratio)
            self.remove_filter_idx = remove_filter_idx[num_prune:]
            if self.remove_filter_idx_history["previous_layer"] is None:
                self.remove_filter_idx_history["previous_layer"] = self.remove_filter_idx
            self.remove_filter_idx_history["current_layer"] = self.remove_filter_idx
        return True

    def remove_conv_filter_kernel(self, conv_name=None, model=None):
        if self.copy_layer.bias is not None:
            w, b = self.base_remove_filter_by_index(
                weight=self.copy_layer.weight.data,
                remove_filter_idx=self.remove_filter_idx_history["current_layer"],
                bias=self.copy_layer.bias.data
            )
            self.copy_layer.weight.data = w
            self.copy_layer.bias.data = b
        else:
            w = self.base_remove_filter_by_index(
                weight=self.copy_layer.weight.data,
                remove_filter_idx=self.remove_filter_idx_history["current_layer"]
            )
            self.copy_layer.weight.data = w
        if conv_name and model and (conv_name in self.conv_to_bn_map):
            bn_name = self.conv_to_bn_map[conv_name]
            bn_layer = get_module_by_name(model, bn_name)
            self.remove_bn_layer(bn_layer, self.remove_filter_idx_history["current_layer"])
        return self.copy_layer

    def remove_bn_layer(self, bn_layer, remove_filter_idx):
        w, b, m, v = self.base_remove_filter_by_index(
            weight=bn_layer.weight.data,
            remove_filter_idx=remove_filter_idx,
            bias=bn_layer.bias.data,
            mean=bn_layer.running_mean.data,
            var=bn_layer.running_var.data
        )
        bn_layer.weight.data = w
        bn_layer.bias.data = b
        bn_layer.running_mean.data = m
        bn_layer.running_var.data = v


In [4]:
class TaylorHook:
    def __init__(self, model, loader, total_layers, total_samples):
        self.model = model
        self.loader = loader
        self.total_layers = total_layers
        self.total_samples = total_samples
        self.f_acts = [None] * total_layers
        self.b_grads = [None] * total_layers

    def attach_hooks(self, conv_layers):
        def fwd_hook(i):
            def hook(module, inp, out):
                val = out.detach().abs().sum(dim=0)
                self.f_acts[i] = val if self.f_acts[i] is None else self.f_acts[i] + val
            return hook
        def bwd_hook(i):
            def hook(module, grad_in, grad_out):
                val = grad_out[0].abs().sum(dim=0)
                self.b_grads[i] = val if self.b_grads[i] is None else self.b_grads[i] + val
            return hook
        for i, conv in enumerate(conv_layers):
            conv.register_forward_hook(fwd_hook(i))
            conv.register_backward_hook(bwd_hook(i))

    def compute(self):
        layers = [m for m in self.model.modules() if isinstance(m, nn.Conv2d)]
        assert len(layers) == self.total_layers
        self.attach_hooks(layers)
        opt = optim.Adam(self.model.parameters())
        loss_fn = nn.CrossEntropyLoss()
        self.model.train()
        for x, y in tqdm(self.loader):
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            out = self.model(x)
            loss_fn(out, y).backward()
        for i in range(self.total_layers):
            self.f_acts[i] /= self.total_samples
            self.b_grads[i] /= self.total_samples

    def get_importance(self, conv_layer, idx):
        fm = self.f_acts[idx]
        gr = self.b_grads[idx]
        imp = (fm * gr).abs().mean(dim=(1,2))
        normed = imp / (imp.norm() + 1e-8)
        return torch.argsort(normed, descending=True)


In [5]:
def get_module_by_name(model, access_string):
    for name in access_string.split("."):
        model = getattr(model, name)
    return model

def set_module_by_name(model, access_string, new_module):
    names = access_string.split(".")
    for name in names[:-1]:
        model = getattr(model, name)
    setattr(model, names[-1], new_module)

def count_nonzero_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    nonzero_params = sum(torch.count_nonzero(p).item() for p in model.parameters())
    compression_ratio = total_params / nonzero_params if nonzero_params else float('inf')
    sparsity = (1 - (nonzero_params / total_params)) * 100
    print(f"Total: {total_params}, Nonzero: {nonzero_params}, Compression: {compression_ratio:.2f}x, Sparsity: {sparsity:.2f}%")
    return compression_ratio, sparsity

def evaluate_model(model, dataloader):
    model.eval()
    correct = top1 = top5 = total = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, pred = out.max(1)
            total += y.size(0)
            correct += pred.eq(y).sum().item()
            _, topk = out.topk(5, 1, True, True)
            top1 += topk[:,0].eq(y).sum().item()
            top5 += torch.any(topk.eq(y.view(-1, 1)), dim=1).sum().item()
            all_preds += pred.cpu().tolist()
            all_labels += y.cpu().tolist()
    print(f"Acc: {100*correct/total:.2f}, Top1: {100*top1/total:.2f}, Top5: {100*top5/total:.2f}")
    print(f"Prec: {precision_score(all_labels, all_preds, average='macro'):.4f}")
    print(f"Recall: {recall_score(all_labels, all_preds, average='macro'):.4f}")
    print(f"F1: {f1_score(all_labels, all_preds, average='macro'):.4f}")
    return all_preds, all_labels

def measure_inference_speed(model, dataloader, device, num_batches=10):
    model.eval(); total_time = 0.0; num_samples = 0
    with torch.no_grad():
        for i, (inputs, _) in enumerate(dataloader):
            if i >= num_batches: break
            inputs = inputs.to(device)
            start_time = time.time()
            _ = model(inputs)
            end_time = time.time()
            total_time += end_time - start_time
            num_samples += inputs.size(0)
    print(f"Infer Speed: {total_time/num_samples:.6f} sec/sample ({num_samples/total_time:.2f} FPS)")


In [6]:
def get_dataset(name):
    if name == "cifar10":
        norm = ((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)); num_classes = 10
        Dataset = torchvision.datasets.CIFAR10
    else:
        norm = ((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)); num_classes = 100
        Dataset = torchvision.datasets.CIFAR100
    tf_train = transforms.Compose([transforms.RandomCrop(32, 4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(*norm)])
    tf_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(*norm)])
    trainset = Dataset(root="./data", train=True, download=True, transform=tf_train)
    testset = Dataset(root="./data", train=False, download=True, transform=tf_test)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
    return trainloader, testloader, trainloader, num_classes


In [7]:
def train_model(model, dataloader, optimizer, criterion, epochs=2):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        start_time = time.time()
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(dataloader):.4f} Time: {time.time()-start_time:.2f}s")
    return model

In [8]:
def evaluate_pipeline(model_name, dataset_name, pretrained_path, one_shot_ratio=0.2, iter_ratio=0.1, iter_steps=2):
    print(f"\n=== {model_name.upper()} on {dataset_name.upper()} ===")
    trainloader, testloader, taylor_loader, num_classes = get_dataset(dataset_name)

    if model_name == "resnet":
        model = ResNet101(num_classes)
    elif model_name == "mobilenet":
        model = MobileNetV2(num_classes)
    else:
        model = VGG(num_classes)

    checkpoint = torch.load(pretrained_path, map_location=device)
    model.load_state_dict(checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint, strict=False)
    model = model.to(device)

    print("\n-- Baseline --")
    evaluate_model(model, testloader)
    count_nonzero_params(model)
    measure_inference_speed(model, testloader, device)

    layer_store = [m for m in model.modules() if isinstance(m, nn.Conv2d)]

    conv_to_bn_map = {}
    prev_conv = None
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d): prev_conv = name
        elif isinstance(module, nn.BatchNorm2d) and prev_conv:
            conv_to_bn_map[prev_conv] = name
            prev_conv = None

    print("\n-- One-Shot Pruning --")
    pe = pruning_engine(
        pruning_method="Taylor",
        pruning_ratio=one_shot_ratio,
        tool_net=model,
        taylor_loader=taylor_loader,
        total_sample_size=len(trainloader.dataset),
        layer_store_private_variable=layer_store,
        conv_to_bn_map=conv_to_bn_map
    )

    for lname in conv_to_bn_map:
        orig_layer = get_module_by_name(model, lname)
        pe.set_layer(orig_layer, main_layer=True)
        masked_layer = pe.remove_conv_filter_kernel(conv_name=lname, model=model)
        set_module_by_name(model, lname, masked_layer)

    evaluate_model(model, testloader)
    count_nonzero_params(model)
    measure_inference_speed(model, testloader, device)

    print("\n-- Iterative Pruning --")
    for it in range(iter_steps):
        print(f"Iter {it+1}")
        pe_iter = pruning_engine(
            pruning_method="Taylor",
            pruning_ratio=iter_ratio,
            tool_net=model,
            taylor_loader=taylor_loader,
            total_sample_size=len(trainloader.dataset),
            layer_store_private_variable=[m for m in model.modules() if isinstance(m, nn.Conv2d)],
            conv_to_bn_map=conv_to_bn_map
        )
        for lname in conv_to_bn_map:
            current_layer = get_module_by_name(model, lname)
            pe_iter.set_layer(current_layer, main_layer=True)
            masked_layer = pe_iter.remove_conv_filter_kernel(conv_name=lname, model=model)
            set_module_by_name(model, lname, masked_layer)
        train_model(model, trainloader, optim.Adam(model.parameters()), nn.CrossEntropyLoss(), epochs=2)
        evaluate_model(model, testloader)
        count_nonzero_params(model)
        measure_inference_speed(model, testloader, device)


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
model_paths = {
    "resnet_cifar100": "/content/drive/MyDrive/Models/Model@ResNet101_CIFAR100.pt",
    "mobilenet_cifar100": "/content/drive/MyDrive/Models/Model@Mobilenetv2_CIFAR100.pt",
    "vgg_cifar100": "/content/drive/MyDrive/Models/Model@VGG16_CIFAR100.pt",
    "mobilenet_cifar10": "/content/drive/MyDrive/Models/Model@Mobilenetv2_CIFAR10.pt",
    "vgg_cifar10": "/content/drive/MyDrive/Models/Model@VGG16_CIFAR10.pt"
}



In [27]:
evaluate_pipeline("mobilenet", "cifar10", model_paths["mobilenet_cifar10"])


=== MOBILENET on CIFAR10 ===

-- Baseline --
Acc: 95.80, Top1: 95.80, Top5: 99.78
Prec: 0.9580
Recall: 0.9580
F1: 0.9579
Total: 2296922, Nonzero: 2296922, Compression: 1.00x, Sparsity: 0.00%
Infer Speed: 0.000229 sec/sample (4376.18 FPS)

-- One-Shot Pruning --


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:55<00:00,  7.09it/s]


Acc: 21.98, Top1: 21.98, Top5: 68.35
Prec: 0.6938
Recall: 0.2198
F1: 0.1606
Total: 2296922, Nonzero: 1837629, Compression: 1.25x, Sparsity: 20.00%
Infer Speed: 0.000258 sec/sample (3874.19 FPS)

-- Iterative Pruning --
Iter 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [01:02<00:00,  6.27it/s]


Epoch [1/2] Loss: 0.4642 Time: 63.64s
Epoch [2/2] Loss: 0.3707 Time: 63.75s
Acc: 86.56, Top1: 86.56, Top5: 99.51
Prec: 0.8692
Recall: 0.8656
F1: 0.8644
Total: 2296922, Nonzero: 1807894, Compression: 1.27x, Sparsity: 21.29%
Infer Speed: 0.000505 sec/sample (1979.41 FPS)
Iter 2


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [01:10<00:00,  5.57it/s]


Epoch [1/2] Loss: 0.3607 Time: 71.68s
Epoch [2/2] Loss: 0.3156 Time: 71.67s
Acc: 88.88, Top1: 88.88, Top5: 99.74
Prec: 0.8898
Recall: 0.8888
F1: 0.8887
Total: 2296922, Nonzero: 1807894, Compression: 1.27x, Sparsity: 21.29%
Infer Speed: 0.000238 sec/sample (4205.77 FPS)


In [34]:
evaluate_pipeline("vgg", "cifar10", model_paths["vgg_cifar10"])


=== VGG on CIFAR10 ===

-- Baseline --
Acc: 95.43, Top1: 95.43, Top5: 99.85
Prec: 0.9544
Recall: 0.9543
F1: 0.9542
Total: 33646666, Nonzero: 33646666, Compression: 1.00x, Sparsity: 0.00%
Infer Speed: 0.000076 sec/sample (13093.66 FPS)

-- One-Shot Pruning --


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:25<00:00, 15.52it/s]


Acc: 51.82, Top1: 51.82, Top5: 87.09
Prec: 0.7430
Recall: 0.5182
F1: 0.5065
Total: 33646666, Nonzero: 30681391, Compression: 1.10x, Sparsity: 8.81%
Infer Speed: 0.000089 sec/sample (11222.22 FPS)

-- Iterative Pruning --
Iter 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:26<00:00, 14.87it/s]


Epoch [1/2] Loss: 0.9084 Time: 29.90s
Epoch [2/2] Loss: 0.5672 Time: 29.91s
Acc: 80.15, Top1: 80.15, Top5: 97.23
Prec: 0.8247
Recall: 0.8015
F1: 0.8033
Total: 33646666, Nonzero: 30681391, Compression: 1.10x, Sparsity: 8.81%
Infer Speed: 0.000125 sec/sample (8031.40 FPS)
Iter 2


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:27<00:00, 14.48it/s]


Epoch [1/2] Loss: 0.5285 Time: 31.12s
Epoch [2/2] Loss: 0.4372 Time: 31.15s
Acc: 84.73, Top1: 84.73, Top5: 98.32
Prec: 0.8580
Recall: 0.8473
F1: 0.8485
Total: 33646666, Nonzero: 30681391, Compression: 1.10x, Sparsity: 8.81%
Infer Speed: 0.000168 sec/sample (5963.51 FPS)


In [11]:
evaluate_pipeline("resnet", "cifar100", model_paths["resnet_cifar100"])


=== RESNET on CIFAR100 ===


100%|██████████| 169M/169M [00:04<00:00, 41.3MB/s]



-- Baseline --
Acc: 83.38, Top1: 83.38, Top5: 96.20
Prec: 0.8353
Recall: 0.8338
F1: 0.8336
Total: 42697380, Nonzero: 42697380, Compression: 1.00x, Sparsity: 0.00%
Infer Speed: 0.000161 sec/sample (6203.07 FPS)

-- One-Shot Pruning --


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [04:35<00:00,  1.42it/s]


Acc: 10.16, Top1: 10.16, Top5: 30.56
Prec: 0.4529
Recall: 0.1016
F1: 0.1084
Total: 42697380, Nonzero: 34120581, Compression: 1.25x, Sparsity: 20.09%


...

Infer Speed: 0.000391 sec/sample (2555.17 FPS)

-- Iterative Pruning --
Iter 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [05:00<00:00,  1.30it/s]


Epoch [1/2] Loss: 2.7295 Time: 306.19s
Epoch [2/2] Loss: 1.9218 Time: 305.97s
Acc: 47.99, Top1: 47.99, Top5: 80.22
Prec: 0.5281
Recall: 0.4799
F1: 0.4706
Total: 42697380, Nonzero: 34127040, Compression: 1.25x, Sparsity: 20.07%
Infer Speed: 0.000350 sec/sample (2854.14 FPS)
Iter 2


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [05:22<00:00,  1.21it/s]


Epoch [1/2] Loss: 1.6798 Time: 328.57s
Epoch [2/2] Loss: 1.4307 Time: 327.96s
Acc: 58.42, Top1: 58.42, Top5: 87.04
Prec: 0.6141
Recall: 0.5842
F1: 0.5779
Total: 42697380, Nonzero: 34127040, Compression: 1.25x, Sparsity: 20.07%
Infer Speed: 0.000938 sec/sample (1065.63 FPS)


In [29]:
evaluate_pipeline("mobilenet", "cifar100", model_paths["mobilenet_cifar100"])


=== MOBILENET on CIFAR100 ===


100%|██████████| 169M/169M [00:04<00:00, 35.0MB/s]



-- Baseline --
Acc: 78.80, Top1: 78.80, Top5: 95.22
Prec: 0.7899
Recall: 0.7880
F1: 0.7878
Total: 2412212, Nonzero: 2412212, Compression: 1.00x, Sparsity: 0.00%
Infer Speed: 0.000167 sec/sample (5998.20 FPS)

-- One-Shot Pruning --


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:54<00:00,  7.17it/s]


Acc: 1.70, Top1: 1.70, Top5: 8.47
Prec: 0.0102
Recall: 0.0170
F1: 0.0041
Total: 2412212, Nonzero: 1952919, Compression: 1.24x, Sparsity: 19.04%


...

Infer Speed: 0.000262 sec/sample (3815.29 FPS)

-- Iterative Pruning --
Iter 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [01:01<00:00,  6.34it/s]


Epoch [1/2] Loss: 1.4515 Time: 63.56s
Epoch [2/2] Loss: 1.1689 Time: 63.51s
Acc: 60.69, Top1: 60.69, Top5: 87.34
Prec: 0.6545
Recall: 0.6069
F1: 0.6062
Total: 2412212, Nonzero: 1925761, Compression: 1.25x, Sparsity: 20.17%
Infer Speed: 0.000271 sec/sample (3694.24 FPS)
Iter 2


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [01:10<00:00,  5.58it/s]


Epoch [1/2] Loss: 1.1311 Time: 71.86s
Epoch [2/2] Loss: 1.0008 Time: 71.75s
Acc: 63.66, Top1: 63.66, Top5: 89.43
Prec: 0.6686
Recall: 0.6366
F1: 0.6363
Total: 2412212, Nonzero: 1925761, Compression: 1.25x, Sparsity: 20.17%
Infer Speed: 0.000414 sec/sample (2414.15 FPS)


In [30]:
evaluate_pipeline("vgg", "cifar100", model_paths["vgg_cifar100"])


=== VGG on CIFAR100 ===

-- Baseline --
Acc: 76.51, Top1: 76.51, Top5: 93.48
Prec: 0.7680
Recall: 0.7651
F1: 0.7653
Total: 34015396, Nonzero: 34015396, Compression: 1.00x, Sparsity: 0.00%
Infer Speed: 0.000066 sec/sample (15048.29 FPS)

-- One-Shot Pruning --


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:25<00:00, 15.35it/s]


Acc: 22.27, Top1: 22.27, Top5: 50.22
Prec: 0.5350
Recall: 0.2227
F1: 0.2380
Total: 34015396, Nonzero: 31050121, Compression: 1.10x, Sparsity: 8.72%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Infer Speed: 0.000078 sec/sample (12853.43 FPS)

-- Iterative Pruning --
Iter 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:26<00:00, 14.91it/s]


Epoch [1/2] Loss: 2.4196 Time: 29.95s
Epoch [2/2] Loss: 1.9217 Time: 30.05s
Acc: 46.88, Top1: 46.88, Top5: 76.85
Prec: 0.5402
Recall: 0.4688
F1: 0.4560
Total: 34015396, Nonzero: 31050121, Compression: 1.10x, Sparsity: 8.72%
Infer Speed: 0.000117 sec/sample (8536.65 FPS)
Iter 2


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 391/391 [00:27<00:00, 14.35it/s]


Epoch [1/2] Loss: 1.8099 Time: 31.25s
Epoch [2/2] Loss: 1.5851 Time: 31.18s
Acc: 55.73, Top1: 55.73, Top5: 83.35
Prec: 0.6122
Recall: 0.5573
F1: 0.5536
Total: 34015396, Nonzero: 31050121, Compression: 1.10x, Sparsity: 8.72%
Infer Speed: 0.000161 sec/sample (6196.05 FPS)
